In [13]:
import re

def extract_test_function_names(filepath):
    pattern = re.compile(r"func (Test\w+)\(")

    test_function_names = []

    with open(filepath, "r") as file:
        content = file.read()

    matches = pattern.findall(content)
    for match in matches:
        test_function_names.append(match)
    return test_function_names

In [14]:
with open("original.txt",'r') as original_file:
    original = original_file.read()

In [15]:
import subprocess

command = [
    "go",
    "test",
    "-run",
    "^TestEqualApprox$",
    "floats/floats_test.go",
    "floats/floats.go",
    "-timeout",
    "10s"
]

rq2_result = {
    "passT" : 0,
    "failT" : 0,
}


def collect_error_1(function_names):
    flag = False 
    for function_name in function_names:
        command[3] = "^" + function_name+ "$"
        result = subprocess.run(command, capture_output=True, text=True).stdout
        if "setup failed" in result or "build failed" in result:
            return False

        if result[:2] != "ok": 
           flag = True
        else:
            rq2_result["failT"] += 1
    return flag      

def collect_error_2(function_names):
    for function_name in function_names:
        command[3] = "^" + function_name+ "$"
        result = subprocess.run(command, capture_output=True, text=True).stdout
        if result[:2] == "ok":
            rq2_result["passT"] += 1


In [16]:
import json
report_file_path = "/Users/maike/Desktop/gonum/reports.json"

with open(report_file_path,'r') as report_file:
    report = json.load(report_file)

map_killed = {}
for killed in report["killed"]:
    map_killed[killed["checksum"]] = killed["mutator"]["mutatedSourceCode"]

In [17]:
with open("/Users/maike/Desktop/llm-test-generation/rq2_completion/boltdb/completion_1.json",'r') as completion_file:
    completion = json.load(completion_file)

In [1]:
from tqdm import tqdm


test_file_path = "/Users/maike/Desktop/gonum/floats/floats_test.go"
source_file_path = "/Users/maike/Desktop/gonum/floats/floats.go" 
        
for checksum in tqdm(completion):
    with open(test_file_path,'w') as test_file:
        test_file.write(completion[checksum])
    
    with open(source_file_path,'w') as source_file:
        source_file.write(map_killed[checksum])
    
    function_names = extract_test_function_names(test_file_path)
    
    r = collect_error_1(function_names)
    
    if r == True:
        with open(source_file_path,'w') as source_file:
            source_file.write(original)
        collect_error_2(function_names)  

rate = rq2_result["passT"]/(rq2_result["passT"] + rq2_result["failT"])
print(f"{rq2_result["passT"]}/{rq2_result["passT"] + rq2_result["failT"]} ({rate})")

SyntaxError: invalid syntax (4194753192.py, line 24)